In [133]:
import requests
from multiprocessing import Process, Manager
import json
import pandas as pd

In [134]:
def get_story(story_id, stories):
    url = 'https://hacker-news.firebaseio.com/v0/item/%d.json' % story_id
    resp = requests.get(url)
    stories.append(resp.json())

In [135]:
def get_top_stories():
    manager = Manager()
    stories = manager.list()
    url = 'https://hacker-news.firebaseio.com/v0/topstories.json'
    ids = requests.get(url)
    processes = [Process(target=get_story, args=(sid, stories))
                 for sid in ids.json()]
    for p in processes:
        p.start()
    for p in processes:
        p.join()
    return stories

In [136]:
df = pd.read_json('../../data/hn.json')

In [137]:
df['id'] = df.index.astype(int)

In [138]:
top_stories = get_top_stories()

In [139]:
new_df = pd.DataFrame([t for t in top_stories])

In [140]:
joined = df.append(new_df)

In [141]:
joined.set_index('id').index

Int64Index([11693122, 11693168, 11693169, 11693184, 11693185, 11693193,
            11693217, 11693232, 11693240, 11693296,
            ...
            11720557, 11723025, 11707493, 11707517, 11706669, 11714068,
            11725369, 11720608, 11709078, 11721609],
           dtype='int64', name=u'id', length=1149)

In [142]:
df = joined.drop_duplicates(subset='id', keep='last')

In [143]:
df.set_index('id').to_json('../../data/hn.json')